 # Bibliotecas

In [ ]:
#Importações de Bases de Dados
from sklearn.datasets import load_iris

#Importação de Esquemas de Avaliação
from sklearn.model_selection import StratifiedKFold

#Importação de Modelos de Classificação
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier


#Importação de Métricas de Avaliação
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

#Importação de Utilitários
import pandas as pd
import numpy as np

# Funções

## Preprocessamento dos Dados

### Mapeando os valores de atributos discretos dado um dicionário criado manualmente

In [ ]:
def preproc_dicionario(df,dicionario):  
  for column in df.columns[:-1]:
    if column in dicionario:
      df[column] = df[column].replace(dicionario[column])

### Mapeando os valores de atributos discretos utilizando LabelEncoders

In [ ]:
def preproc_enconder(df):
  dict_encoders = {}
  for column in df.columns[:-1]:
    encoder_column = LabelEncoder()
    df[column] = encoder_column.fit_transform(df[column])
    dict_encoders[column] = encoder_column
  
  return dict_encoders

## Avaliação do Modelo de Classificação

In [ ]:
def evaluate(y_test, y_pred):
  dict_results = {}
  dict_results['Acurácia'] = accuracy_score(y_test, y_pred)
  dict_results['Micro Revocação'] = recall_score(y_test, y_pred, average='micro')
  dict_results['Macro Revocação'] = recall_score(y_test, y_pred, average='macro')
  dict_results['Micro Precisao'] = precision_score(y_test, y_pred, average='micro')
  dict_results['Macro Precisao'] = precision_score(y_test, y_pred, average='macro')
  dict_results['Micro F1'] = f1_score(y_test,y_pred, average='micro')
  dict_results['Macro F1'] = f1_score(y_test,y_pred, average='macro')
  return dict_results

In [ ]:
def results_k_fold_cross_validation(classifier, X, y, n_splits=10, shuffle=True, random_state = 0):

  resultados = pd.DataFrame(columns=['Iteração','Acurácia','Micro Revocação','Macro Revocação','Micro Precisao','Macro Precisao','Micro F1','Macro F1'])

  kf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state = random_state)
  contador = 1
  for train_index, test_index in kf.split(X,y):
    # Separando o conjunto de treino e teste
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]

    #Construindo um modelo de classificação
    classifier.fit(X_train,y_train)

    #Avaliando um modelo de classificação com diferentes medidas
    y_pred = classifier.predict(X_test)
    dict_results = evaluate(y_test, y_pred)
    dict_results['Iteração'] = contador
    contador = contador + 1
    resultados = resultados.append(dict_results, ignore_index=True)
  
  return resultados

# Carregando as Bases

## Caregando a Weather

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Datasets/Toy/weather.nominal.csv')

### Conversão de valores discretos para numéricos utilizando um mapeamento manualmente definido

In [ ]:
dic_conversao = {'outlook': {'sunny':0,'overcast':1,'rainy':2}, 
                 'temperature': {'cool':0,'mild':1,'hot':2}, 
                 'humidity': {'normal':0,'high':1}, 
                 'windy': {False:0,True:1}}
preproc_dicionario(df,dic_conversao)

### Conversão de valores discretos para numéricos utilizando um LabelEnconder

In [ ]:
dict_encoders = preproc_enconder(df)

### Separando os Exemplos e os Rótulos

In [ ]:
data = np.array(df)
X = data[:,:-1]
y = data[:,-1]

## Carregando a Iris

In [ ]:
data = load_iris()

### Separando os Exemplos e os Rótulos

In [ ]:
X = data['data']
y = data['target']
class_names = data['target_names']

#Área de Testes

In [ ]:
resultados = results_k_fold_cross_validation(MLPClassifier(), X, y, n_splits=4)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
resultados

,Iteração,Acurácia,Micro Revocação,Macro Revocação,Micro Precisao,Macro Precisao,Micro F1,Macro F1
0,1.0,0.894737,0.894737,0.888889,0.894737,0.921569,0.894737,0.888889
1,2.0,0.894737,0.894737,0.888889,0.894737,0.921569,0.894737,0.888889
2,3.0,0.837838,0.837838,0.846154,0.837838,0.888889,0.837838,0.833333
3,4.0,0.972973,0.972973,0.974359,0.972973,0.974359,0.972973,0.973333


In [ ]:
resultados.describe()

,Iteração,Acurácia,Micro Revocação,Macro Revocação,Micro Precisao,Macro Precisao,Micro F1,Macro F1
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,2.500000,0.900071,0.900071,0.899573,0.900071,0.926596,0.900071,0.896111
std,1.290994,0.055511,0.055511,0.053774,0.055511,0.035373,0.055511,0.057760
min,1.000000,0.837838,0.837838,0.846154,0.837838,0.888889,0.837838,0.833333
25%,1.750000,0.880512,0.880512,0.878205,0.880512,0.913399,0.880512,0.875000
50%,2.500000,0.894737,0.894737,0.888889,0.894737,0.921569,0.894737,0.888889
75%,3.250000,0.914296,0.914296,0.910256,0.914296,0.934766,0.914296,0.910000
max,4.000000,0.972973,0.972973,0.974359,0.972973,0.974359,0.972973,0.973333
